# Natural Robustness Toolkit (NRTK) demo

This notebook contains an end-to-end demostration of Dioptra that can be run on any modern laptop.

## Setup

Below we import the necessary Python modules and ensure the proper environment variables are set so that all the code blocks will work as expected.

In [1]:
# Import packages from the Python standard library
import importlib.util
import os
import sys
import pprint
import time
import warnings
from pathlib import Path


def register_python_source_file(module_name: str, filepath: Path) -> None:
    """Import a source file directly.

    Args:
        module_name: The module name to associate with the imported source file.
        filepath: The path to the source file.

    Notes:
        Adapted from the following implementation in the Python documentation:
        https://docs.python.org/3/library/importlib.html#importing-a-source-file-directly
    """
    spec = importlib.util.spec_from_file_location(module_name, str(filepath))
    module = importlib.util.module_from_spec(spec)
    sys.modules[module_name] = module
    spec.loader.exec_module(module)


# Filter out warning messages
warnings.filterwarnings("ignore")

# Experiment name
EXPERIMENT_NAME = "pytorch_maite_nrtk"

# Default address for accessing the RESTful API service
RESTAPI_ADDRESS = "http://localhost:80"

# Set DIOPTRA_RESTAPI_URI variable if not defined, used to connect to RESTful API service
os.environ["DIOPTRA_RESTAPI_URI"] = RESTAPI_ADDRESS

# Default address for accessing the MLFlow Tracking server
MLFLOW_TRACKING_URI = "http://localhost:35000"

# Set MLFLOW_TRACKING_URI variable, used to connect to MLFlow Tracking service
if os.getenv("MLFLOW_TRACKING_URI") is None:
    os.environ["MLFLOW_TRACKING_URI"] = MLFLOW_TRACKING_URI

# Path to workflows archive
WORKFLOWS_TAR_GZ = Path("workflows.tar.gz")

# Register the examples/scripts directory as a Python module
register_python_source_file("scripts", Path("..", "scripts", "__init__.py"))

from scripts.client import DioptraClient
from scripts.utils import make_tar

# Import third-party Python packages
import numpy as np
from mlflow.tracking import MlflowClient

# Create random number generator
rng = np.random.default_rng(54399264723942495723666216079516778448)

## Submit and run jobs

The entrypoints that we will be running in this example are implemented in the Python source files under `src/` and the `src/MLproject` file.
To run these entrypoints within Dioptra's architecture, we need to package those files up into an archive and submit it to the Dioptra RESTful API to create a new job.
For convenience, we provide the `make_tar` helper function defined in `examples/scripts/utils.py`.

In [2]:
def mlflow_run_id_is_not_known(response_nrtk):
    return response_nrtk["mlflowRunId"] is None and response_nrtk["status"] not in [
        "failed",
        "finished",
    ]

In [3]:
make_tar(["src"], WORKFLOWS_TAR_GZ)

PosixPath('/Users/bhodges/Desktop/Dioptra branches/bjpatrick-dioptra-nrtk/examples/pytorch-obj-detect-nrtk/workflows.tar.gz')

To connect with the endpoint, we will use a client class defined in the `examples/scripts/client.py` file that is able to connect with the Dioptra RESTful API using the HTTP protocol.
We connect using the client below.
The client uses the environment variable `DIOPTRA_RESTAPI_URI`, which we configured at the top of the notebook, to figure out how to connect to the Dioptra RESTful API.

In [4]:
restapi_client = DioptraClient()

We need to register an experiment under which to collect our job runs.
The code below checks if the relevant experiment exists.
If it does, then it just returns info about the experiment, if it doesn't, it then registers the new experiment.

In [5]:
!python ../scripts/register_task_plugins.py --force --plugins-dir ../task-plugins --api-url http://localhost:80

╭─────────────────────────────────────────────────╮
│ Dioptra Examples - Register Custom Task Plugins │
╰─────────────────────────────────────────────────╯
 ‣ plugins_dir: ../task-plugins
 ‣ api_url: http://localhost:80
 ‣ force: True
 ✔ Overwritten. Removed and re-registered the custom task plugin 'pytorch_mi'.
 ✔ Overwritten. Removed and re-registered the custom task plugin 
'model_inversion'.
 ✔ Overwritten. Removed and re-registered the custom task plugin 'modelscan'.
 ✔ Overwritten. Removed and re-registered the custom task plugin 
'custom_poisoning_plugins'.
 ✔ Overwritten. Removed and re-registered the custom task plugin 
'custom_fgm_plugins'.
 ✔ Overwritten. Removed and re-registered the custom task plugin 
'pixel_threshold'.
 ✔ Overwritten. Removed and re-registered the custom task plugin 
'custom_patch_plugins'.
 ✔ Overwritten. Removed and re-registered the custom task plugin 'maite'.
 ✔ Overwritten. Removed and re-registered the custom task plugin 'pytorch_d2'.
 ✔ Overwritte

In [6]:
response_experiment = restapi_client.get_experiment_by_name(name=EXPERIMENT_NAME)

if response_experiment is None or "Not Found" in response_experiment.get("message", []):
    response_experiment = restapi_client.register_experiment(name=EXPERIMENT_NAME)

response_experiment

{'experimentId': 1,
 'createdOn': '2024-07-13T18:27:57.486712',
 'lastModified': '2024-07-13T18:27:57.486712',
 'name': 'pytorch_maite_nrtk'}

The `full_workflow` entry point tests basic MAITE functionality: load a dataset from huggingface, load a model from huggingface, load a metric from torchvision and run that metric on that model/dataset. It also saves the model into MLFlow.

In [ ]:
response_test_metrics = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="full_workflow",
    entry_point_kwargs=" ".join([
        "-P dataset_provider_name=torchvision",
        "-P dataset_name=voc",
        "-P dataset_task=object-detection",
        "-P split=val",
        "-P model_provider_name=torchvision",
        "-P model_name=fasterrcnn_resnet50_fpn",
        "-P model_task=object-detection",
        "-P metric_provider_name=torchmetrics",
        "-P metric_name=MeanAveragePrecision",
        "-P metric_task=detection",
        "-P classes=46",
        "-P batch_size=4",
        "-P shape=[800,800]"
    ]),
    queue="pytorch_cpu",
    timeout="1h",
)
pprint.pprint(response_test_metrics)

The `save_model` entry point loads a model from huggingface and saves it to MLFlow. In this example, we are pulling this object detection model from huggingface: https://huggingface.co/spaces/lkeab/transfiner/tree/main 

In [7]:
response_model = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="save_model",
    entry_point_kwargs=" ".join([
        "-P model_provider_name=torchvision",
        "-P model_name=fasterrcnn_resnet50_fpn",
        "-P model_task=object-detection"
    ]),
    queue="pytorch_cpu",
    timeout="1h",
)
pprint.pprint(response_model)

{'createdOn': '2024-07-16T20:41:05.733304',
 'dependsOn': None,
 'entryPoint': 'save_model',
 'entryPointKwargs': '-P model_provider_name=torchvision -P '
                     'model_name=fasterrcnn_resnet50_fpn -P '
                     'model_task=object-detection',
 'experimentId': 1,
 'jobId': 'd77bda2f-a610-4bf5-a6c1-4788205eb880',
 'lastModified': '2024-07-16T20:41:05.733304',
 'mlflowRunId': None,
 'queueId': 3,
 'status': 'queued',
 'timeout': '1h',
 'workflowUri': 's3://workflow/4340a0e2bd5e4901adeccd9de152bc5b/workflows.tar.gz'}


The `scan_model` entrypoint loads the previously saved model from MLFlow and uses modelscan to evaluate the model files for insecure or malicious code.

It is important to note that modelscan searches for publicly known vulnerable coding practices, which the tool labels as a CRITICAL vulnerability. If the tool is used to scan models on opensource platforms like HuggingFace, there is a possibility for the tool to report a critical finding. Use caution when engaging with models that produce critical scan reports and ensure the model is published by a trusted source. 

In [8]:
while mlflow_run_id_is_not_known(response_model):
    time.sleep(1)
    response_model = restapi_client.get_job_by_id(response_model["jobId"])

response_scan_model = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="scan_model",
    entry_point_kwargs=" ".join([
        f"-P mlflow_run_id={response_model['mlflowRunId']}",
    ]),
    queue="pytorch_cpu",
    timeout="1h",
    depends_on=response_model["jobId"],
)
pprint.pprint(response_scan_model)

{'createdOn': '2024-07-16T20:44:08.019447',
 'dependsOn': 'd77bda2f-a610-4bf5-a6c1-4788205eb880',
 'entryPoint': 'scan_model',
 'entryPointKwargs': '-P mlflow_run_id=fe5b0db1fee14cb294e8c1c87acf7720',
 'experimentId': 1,
 'jobId': 'f4b45e7f-f60e-4004-8a72-a95d119d66db',
 'lastModified': '2024-07-16T20:44:08.019447',
 'mlflowRunId': None,
 'queueId': 3,
 'status': 'queued',
 'timeout': '1h',
 'workflowUri': 's3://workflow/1d40de8c18b649ddb230a2d88ebf9a99/workflows.tar.gz'}


The `test_model` entrypoint loads the previously saved model from MLFlow into a MAITE-readable format, and then uses maite to test metrics and a dataset on it.

In [9]:
while mlflow_run_id_is_not_known(response_model):
    time.sleep(1)
    response_model = restapi_client.get_job_by_id(response_model["jobId"])

response_use_model = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="test_model",
    entry_point_kwargs=" ".join([
        "-P model_name=loaded_model",
        "-P model_version=1",
        "-P model_task=object-detection", 
        "-P dataset_provider_name=huggingface",
        "-P dataset_name=detection-datasets/fashionpedia",
        "-P dataset_task=object-detection",
        "-P split=val",
        "-P metric_provider_name=torchmetrics",
        "-P metric_name=MeanAveragePrecision", 
        "-P metric_task=detection",
        "-P classes=80",
        "-P batch_size=4",
        "-P shape=[800,800]"
    ]),
    queue="pytorch_cpu",
    timeout="1h",
)
#HuggingFace datasets:
#detection-datasets/fashionpedia; classes=46; TEST STATUS=Success
#detection-datasets/coco; classes=80; TEST STATUS=Success

The `load_dataset` entrypoint loads a dataset from disk, puts it into maite format, then loads a model and metric using maite and runs it on that dataset.

In [ ]:
response_load_dataset = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="load_dataset",
    entry_point_kwargs=" ".join([
        "-P subset=400"
    ]),
    queue="pytorch_cpu",
    timeout="1h",
)

The `gen_nrtk` entrypoint loads a dataset using MAITE, applies a NRTK perturbations on it, creates a new `perturbed_dataset` and registers it to MLFlow as an artifact. 

Currently, Dioptra's NRTK custom plugin is configured to apply skimage and PIL perturbations to object detection datasets. For more insight into NRTK's perturbations, please refer to their documentation. Parameters for this example were derived from NRTK's perturbation jupyter notebook example here: https://github.com/Kitware/nrtk/blob/main/examples/perturbers.ipynb.

In [15]:
response_gen_nrtk = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="gen_nrtk",
    entry_point_kwargs=" ".join([
        "-P dataset_provider_name=huggingface",
        "-P dataset_name=detection-datasets/fashionpedia",
        "-P dataset_task=object-detection",
        "-P split=val",
        "-P perturbation=SaltNoisePerturber",
        "-P seed=42",
        "-P amount=0.25"
    ]),
    queue="pytorch_cpu",
    timeout="1h",
)
#Tested datasets
#dataset_provider_name=huggingface; dataset_name=detection-datasets/coco
#dataset_provider_name=huggingface; dataset_name=detection-datasets/fashionpedia

The `infer_nrtk` entrypoint takes the previously generated `perturbed_dataset` results and runs it against a given model and metric. It is included here as a function and tested against 4 models on huggingface from different authors. Note that not all targeted models on huggingface are compatible for various reasons - missing `config.json`, different requirements for data formatting, etc. The examples included below worked at the time of testing.

Although MAITE supports torchvision as a provider as well, torchvision does not seem to provide pretrained CIFAR10 models. An ImageNET example may be more suited to cross-testing torchvision and huggingface models.

In [13]:
def test_nrtk_dataset(provider, model):
    global response_gen_nrtk
    while mlflow_run_id_is_not_known(response_gen_nrtk):
        time.sleep(1)
        response_gen_nrtk = restapi_client.get_job_by_id(response_gen_nrtk["jobId"])
    response_infer_nrtk = restapi_client.submit_job(
        workflows_file=WORKFLOWS_TAR_GZ,
        experiment_name=EXPERIMENT_NAME,
        entry_point="infer_nrtk",
        entry_point_kwargs=" ".join([
            f"-P run_id={response_gen_nrtk['mlflowRunId']}",
            f"-P model_provider_name={provider}",
            f"-P model_name={model}",
            f"-P model_task=image-classification"
        ]),
        queue="pytorch_cpu",
        timeout="1h",
        depends_on=response_gen_nrtk["jobId"],
    )
    return response_infer_nrtk

In [14]:
test_perturbed_dataset_nrtk("torchvision","fasterrcnn_resnet50_fpn")

{'jobId': '005bdf2e-ec0f-4947-a3eb-c13d8d4a82ed',
 'mlflowRunId': None,
 'experimentId': 1,
 'queueId': 3,
 'createdOn': '2024-07-16T22:13:32.531231',
 'lastModified': '2024-07-16T22:13:32.531231',
 'timeout': '1h',
 'workflowUri': 's3://workflow/cfb8540537e84152a3f8d1189ed92c6c/workflows.tar.gz',
 'entryPoint': 'infer_nrtk',
 'entryPointKwargs': '-P run_id=171fdfd812a848278e5d35bab2960e72 -P model_provider_name=torchvision -P model_name=fasterrcnn_resnet50_fpn -P model_task=object-detection',
 'dependsOn': 'a32b9497-2f92-4112-9f2c-da54ff4d7521',
 'status': 'queued'}

In [ ]:
test_perturbed_dataset_nrtk("huggingface","facebook/detr-resnet-50")